In [ ]:
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import polyscope as ps  # https://polyscope.run/py/
import numpy as np
from tqdm import tqdm
from colour import wavelength_to_XYZ

from pathlib import Path

In [ ]:
from chromalab.observer import Observer, Cone, transformToChromaticity, getSampledHyperCube, getSampledHyperCubeSurface
from chromalab.spectra import Illuminant
from chromalab.adaptation import dead_leaves

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [ ]:
# Tetracolor sampling
# wavelengths = np.arange(390, 830, 10)
# d65 = Illuminant.get("E")

# tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, illuminant=d65, verbose=True)
# smql, rgbs = tetrachromat.get_full_colors_in_activations()
# np.savetxt('./data/tetrachromat_smql.csv', smql, delimiter=',')

In [ ]:
smql = np.loadtxt('./data/tetrachromat_smql.csv', delimiter=',')

In [ ]:
# Initialize polyscope and set some scene options
ps.init()

ps.set_build_gui(False)
ps.set_ground_plane_mode("none")
ps.set_SSAA_factor(4)
ps.set_background_color([0, 0, 0])

image_res = 512
ps.set_window_resizable(True)
ps.set_window_size(image_res / 2, image_res / 2)

In [ ]:
# Set up directories for saving outputs
Path('./out/tri').mkdir(parents=True, exist_ok=True)
Path('./out/RG1G2B').mkdir(parents=True, exist_ok=True)
Path('./out/LMS_Q').mkdir(parents=True, exist_ok=True)
Path('./out/four_gray').mkdir(parents=True, exist_ok=True)

In [ ]:
n_channels = 4

# Trichromat dead leaves
if n_channels == 3:
    # Generate some RGB colors
    hues = np.linspace(0, 0.9, 100)
    color_palette = np.array([mcolors.hsv_to_rgb([hue, 1, 1]) for hue in hues])

    # Set up the dead leaves scene
    dead_leaves(0.5, color_palette, max_iters=20, shape_mode='mixed')

    # Take a screenshot
    dir = './out/tri/'
    ps.screenshot(dir + 'dead_leaves_tri.png')

    # Write a video file
    fd = ps.open_video_file(dir + 'dead_leaves_tri.mp4', fps=60)
    for i in range(60):
        # Add in some camera motion here per frame
        ps.write_video_frame(fd)
    ps.close_video_file(fd)

In [ ]:
# Tetrachromat dead leaves
def animate_leaves(leaves, directions):
    for i in range(len(leaves)):
        leaf = leaves[i]
        direction = directions[i]
        leaf.translate(direction)        

if n_channels == 4:
    # Generate some tetra colors
    # hues = np.linspace(0, 0.9, 100)
    # color_palette = np.array([mcolors.hsv_to_rgb([hue, 1, 1]) for hue in hues])
    # Qs = np.random.rand(color_palette.shape[0], 1)
    # color_palette = np.hstack([color_palette, Qs])

    # Use tetracolors sampled above
    color_palette = smql

    # Set up the dead leaves scene
    leaves = dead_leaves(0.1, color_palette, max_iters=150, shape_mode='mixed')
    
    """
    save_image_mode -- if rasterizing a tetracolor scene, mode to save the output (to either image or video file)
        'LMS_Q' : saves first 3 channels in one RGB .png file, saves 4th channel in one grayscale .png file
        'four_gray' : saves each channel to a separate grayscale .png file
        'RG1G2B' : saves 4 color channels in one RGBA .png file
    """
    tetra_save_mode = 'LMS_Q'
    # tetra_save_mode = 'four_gray'
    # tetra_save_mode = 'RG1G2B'

    dir = f'./out/{tetra_save_mode}/'

    # Take a tetra screenshot
    ps.rasterize_tetra(dir + 'dead_leaves_tetra.png', save_image_mode=tetra_save_mode)

    # Write a tetra video file
    # 'RG1G2B' is a valid save_image_mode for writing video files, but mp4 won't write transparency
    # Most likely want to use 'LMS_Q' or 'four_gray' mode for writing tetra video files!
    tfds = ps.open_tetra_video_file(dir + 'dead_leaves_tetra.mp4', fps=30, save_image_mode=tetra_save_mode)
    
    # Generate random directions for the shapesto move in
    directions = np.random.randn(len(leaves), 2)
    directions = directions / np.linalg.norm(directions, axis=1, keepdims=True)
    directions = np.hstack([directions, np.zeros((len(leaves), 1))])

    for i in range(300):
        animate_leaves(leaves, directions)
        ps.write_tetra_video_frame(tfds)
    ps.close_tetra_video_file(tfds)